In [2]:
import pandas as pd
df = pd.read_csv(r"D:\Desktop/gmail-api-automation/artifacts/for_training.csv")

df = df[['text','lable']]

In [9]:
import sklearn
print(sklearn.__version__)


1.8.0


In [10]:
import sys
sys.path.append(r"D:\Desktop\gmail-api-automation")

import joblib
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# -------- Load Artifacts --------
model = joblib.load(r"D:/Desktop/gmail-api-automation/artifacts/model.pkl")
preprocessor = joblib.load(r"D:\Desktop/gmail-api-automation/artifacts/preprocessor.pkl")
label_encoder = joblib.load(r"D:/Desktop/gmail-api-automation/artifacts/lable_encoder.pkl")

# -------- Prepare Data --------
X = df["text"]
y_true = df["lable"]   # make sure column name matches exactly

# Encode true labels
y_true_encoded = label_encoder.transform(y_true)

# -------- Transform Text --------
X_transformed = preprocessor.transform(X)

# -------- Predict --------
y_pred_encoded = model.predict(X_transformed)

# Decode predictions back to original labels
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# -------- Evaluation --------
print("Accuracy:", accuracy_score(y_true_encoded, y_pred_encoded))
print("\nConfusion Matrix:\n", confusion_matrix(y_true_encoded, y_pred_encoded))
print("\nClassification Report:\n", classification_report(y_true_encoded, y_pred_encoded))


AttributeError: Can't get attribute '_RemainderColsList' on <module 'sklearn.compose._column_transformer' from 'd:\\Desktop\\gmail-api-automation\\gmailapi\\Lib\\site-packages\\sklearn\\compose\\_column_transformer.py'>

In [ ]:
"""import pandas as pd

# Load files
df_all = pd.read_csv("D:\Desktop\gmail-api-automation/artifacts/unlabel.csv")
df_manual = pd.read_csv("D:\Desktop\gmail-api-automation/artifacts/training_emails.csv")

# Remove emails from 3800 that appear in 1800
df_remaining = df_all[
    ~df_all["id"].isin(df_manual["id"])
]

print("Remaining emails:", len(df_remaining))

# Save
df_remaining.to_csv("data/unlabeled_from_3800.csv", index=False)
"""

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\amara\AppData\Local\Temp\ipykernel_5296\2785808624.py:4: SyntaxWarning: invalid escape sequence '\D'
  df_all = pd.read_csv("D:\Desktop\gmail-api-automation/artifacts/unlabel.csv")
C:\Users\amara\AppData\Local\Temp\ipykernel_5296\2785808624.py:5: SyntaxWarning: invalid escape sequence '\D'
  df_manual = pd.read_csv("D:\Desktop\gmail-api-automation/artifacts/training_emails.csv")


Remaining emails: 2000


In [ ]:
"""import sys
import os
from dataclasses import dataclass

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from src.exception import CustomException
from src.logger import logging

from src.utils import clean_text, get_vectorizer


@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path=os.path.join('artifacts','preprocessor.pkl')


class DataTransformation:
    def __init__(self):
        self.data_transformation_config = DataTransformationConfig

        def get_data_transform_object(self):
            try:
                
                text_pipeline = Pipeline(
                    steps=[
                        ("clean_text", clean_text()),
                        ("tfidf", get_vectorizer())
                    ]
                )

                logging.info("Cleaning text and vectoring")
                return text_pipeline

            except Exception as e:
                raise CustomException(e,sys)
            

def initiate_data_transformation(self, train_path, test_path):
    try:
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(train_path)

        logging.info("Read train and test data completed")

        logging.info("Obtaining preprocessing object")

        preprocessor_obj = self.get_data_transformer_object()

        target_column_name = "label"
        numerical_columns = "text"

        input_feature_train_df = train_df.drop(columns=[target_column_name], axis=1)
        target_feature_train_df = train_df[target_column_name]

        input_feature_test_df = test_df.drop(columns=[target_column_name], axis=1)
        target_feature_test_df = test_df[target_column_name]

        logging.info(f"applying preprocessing object in training and test dataframe")

        input_feature_train_arr = preprocessor_obj.fit_transform(input_feature_train_df)
        input_feature_test_arr = preprocessor_obj.transform(input_feature_test_df)

        train_arr = np.c_[
            input_feature_train_arr, np.array(target_feature_train_df)
        ]
        test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

        logging.info("saving preprocessing object")

        save_object(
            file_path = self.data_transformation_config.preprocessor_obj_file_path,
            obj = preprocessor_obj
        )

        return (
            train_arr,
            test_arr,
            self.data_transformation_config.preprocessor_obj_file_path,
        )
    except Exception as e:
        raise CustomException(e, sys)"""

In [ ]:
"""## to add samples to label.csv to manually label emails so that model get trained on more data.
import pandas as pd
import numpy as np

# -----------------------
# CONFIG
# -----------------------
LABEL_FILE = "label.csv"
UNLABEL_FILE = "unlabel.csv"
ADD_TO_LABEL = 300
RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)

# -----------------------
# LOAD FILES
# -----------------------
label_df = pd.read_csv(LABEL_FILE)
unlabel_df = pd.read_csv(UNLABEL_FILE)

print("Before update")
print("label.csv   :", len(label_df))
print("unlabel.csv :", len(unlabel_df))

# -----------------------
# SAMPLE 200 FROM UNLABELED
# -----------------------
new_labeled = unlabel_df.sample(
    n=ADD_TO_LABEL,
    random_state=RANDOM_STATE
)

# -----------------------
# UPDATE LABELED DATA
# -----------------------
updated_label_df = pd.concat(
    [label_df, new_labeled],
    ignore_index=True
)

# -----------------------
# UPDATE UNLABELED DATA
# -----------------------
updated_unlabel_df = unlabel_df.drop(
    index=new_labeled.index
).reset_index(drop=True)

# -----------------------
# SAVE BACK
# -----------------------
updated_label_df.to_csv("label.csv", index=False)
updated_unlabel_df.to_csv("unlabel.csv", index=False)

# -----------------------
# FINAL CHECK
# -----------------------
print("\nAfter update")
print("label.csv   :", len(updated_label_df))   # should be 800
print("unlabel.csv :", len(updated_unlabel_df))
print("DONE")
"""

Before update
label.csv   : 1500
unlabel.csv : 2300

After update
label.csv   : 1800
unlabel.csv : 2000
DONE


In [19]:
import pandas as pd

In [31]:

df = pd.read_csv("data\label.csv")
df2 = pd.read_csv("gmail_last_month_label_ready.csv")
print(df.isnull().sum())
df.isnull().sum()

id           0
from         0
subject      1
date         0
snippet      3
text         0
label      203
dtype: int64


<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\amara\AppData\Local\Temp\ipykernel_2324\119929104.py:1: SyntaxWarning: invalid escape sequence '\l'
  df = pd.read_csv("data\label.csv")


id           0
from         0
subject      1
date         0
snippet      3
text         0
label      203
dtype: int64

In [29]:
# Combine both
merged_df = pd.concat([df, df2], ignore_index=True)

# Remove duplicates based on email id
# Keep old rows first (existing labels are preserved)
merged_df = merged_df.drop_duplicates(subset="id", keep="first")

# Save back to label.csv (UPDATE)
merged_df.to_csv(r"data/label.csv", index=False)

print("label.csv updated successfully")

# Optional sanity check
print(merged_df.isnull().sum())

label.csv updated successfully
id           0
from         0
subject      1
date         0
snippet      3
text         0
label      203
dtype: int64


In [32]:
df.shape

(2003, 7)

In [33]:
df2.shape

(209, 7)

<StringArray>
['PROMOTION', 'TRASH', 'IMPORTANT', 'OFFICIAL', 'INFORMATION']
Length: 5, dtype: str

In [6]:
df["domain"] = df["from"].str.extract(r'@([\w.-]+)')


In [7]:
df.head(3)

,id,from,subject,date,snippet,text,label,domain
0,199ead51e3bab605,GoDaddy <email@e.godaddy.com>,.IN and .CO.IN domains for just ₹299!,2025-10-16 02:20:02+00:00,"Act fast, end of year savings are only availab...",.IN and .CO.IN domains for just ₹299! Act fast...,PROMOTION,e.godaddy.com
1,191f0d4f885c68bb,AJIO <alert@mailer.ajio.in>,Expiring Soon: AJIO Perks Worth 200 Points,2024-09-14 13:59:32+00:00,Don&#39;t miss shopping at the AJIO All Stars ...,Expiring Soon: AJIO Perks Worth 200 Points Don...,PROMOTION,mailer.ajio.in
2,194fbf2b18676a62,Google AI Studio <googleaistudio-noreply@googl...,"Varun reddy, design prompts with Google AI Studio",2025-02-12 20:56:13+00:00,Craft compelling prompts for powerful AI resul...,"Varun reddy, design prompts with Google AI Stu...",PROMOTION,google.com


In [8]:
df["text"] = (
    df["domain"] + " " +
    df["text"]
)


In [9]:
df.head(3)

,id,from,subject,date,snippet,text,label,domain
0,199ead51e3bab605,GoDaddy <email@e.godaddy.com>,.IN and .CO.IN domains for just ₹299!,2025-10-16 02:20:02+00:00,"Act fast, end of year savings are only availab...",e.godaddy.com .IN and .CO.IN domains for just ...,PROMOTION,e.godaddy.com
1,191f0d4f885c68bb,AJIO <alert@mailer.ajio.in>,Expiring Soon: AJIO Perks Worth 200 Points,2024-09-14 13:59:32+00:00,Don&#39;t miss shopping at the AJIO All Stars ...,mailer.ajio.in Expiring Soon: AJIO Perks Worth...,PROMOTION,mailer.ajio.in
2,194fbf2b18676a62,Google AI Studio <googleaistudio-noreply@googl...,"Varun reddy, design prompts with Google AI Studio",2025-02-12 20:56:13+00:00,Craft compelling prompts for powerful AI resul...,"google.com Varun reddy, design prompts with Go...",PROMOTION,google.com


In [10]:
df['text'].head()

0    e.godaddy.com .IN and .CO.IN domains for just ...
1    mailer.ajio.in Expiring Soon: AJIO Perks Worth...
2    google.com Varun reddy, design prompts with Go...
3    codewithmosh.com 🚨 Final Hours – Sale Ends Ton...
4    hackerrankmail.com Improve your Coding Skills ...
Name: text, dtype: str

In [11]:
df = df.drop(columns=["domain"])


In [12]:
df.head()

,id,from,subject,date,snippet,text,label
0,199ead51e3bab605,GoDaddy <email@e.godaddy.com>,.IN and .CO.IN domains for just ₹299!,2025-10-16 02:20:02+00:00,"Act fast, end of year savings are only availab...",e.godaddy.com .IN and .CO.IN domains for just ...,PROMOTION
1,191f0d4f885c68bb,AJIO <alert@mailer.ajio.in>,Expiring Soon: AJIO Perks Worth 200 Points,2024-09-14 13:59:32+00:00,Don&#39;t miss shopping at the AJIO All Stars ...,mailer.ajio.in Expiring Soon: AJIO Perks Worth...,PROMOTION
2,194fbf2b18676a62,Google AI Studio <googleaistudio-noreply@googl...,"Varun reddy, design prompts with Google AI Studio",2025-02-12 20:56:13+00:00,Craft compelling prompts for powerful AI resul...,"google.com Varun reddy, design prompts with Go...",PROMOTION
3,19547f8cee67ac3b,Mosh Hamedani <support@codewithmosh.com>,🚨 Final Hours – Sale Ends Tonight!,2025-02-27 15:14:03+00:00,Last Call! Sale Ends in a Few Hours ‌ ‌ ‌ ‌ ‌ ...,codewithmosh.com 🚨 Final Hours – Sale Ends Ton...,PROMOTION
4,195ce018993e9741,HackerRank Team <no-reply@hackerrankmail.com>,"Improve your Coding Skills with Say ""Hello, Wo...",2025-03-25 15:52:41+00:00,"Hi Varun reddy, Improve your skills with this ...",hackerrankmail.com Improve your Coding Skills ...,PROMOTION


In [13]:
df.to_csv("label.csv", index=False)


In [16]:
dd = pd.read_csv("data/label.csv")

In [17]:
dd.head()

,id,from,subject,date,snippet,text,label
0,199ead51e3bab605,GoDaddy <email@e.godaddy.com>,.IN and .CO.IN domains for just ₹299!,2025-10-16 02:20:02+00:00,"Act fast, end of year savings are only availab...",e.godaddy.com .IN and .CO.IN domains for just ...,PROMOTION
1,191f0d4f885c68bb,AJIO <alert@mailer.ajio.in>,Expiring Soon: AJIO Perks Worth 200 Points,2024-09-14 13:59:32+00:00,Don&#39;t miss shopping at the AJIO All Stars ...,mailer.ajio.in Expiring Soon: AJIO Perks Worth...,PROMOTION
2,194fbf2b18676a62,Google AI Studio <googleaistudio-noreply@googl...,"Varun reddy, design prompts with Google AI Studio",2025-02-12 20:56:13+00:00,Craft compelling prompts for powerful AI resul...,"google.com Varun reddy, design prompts with Go...",PROMOTION
3,19547f8cee67ac3b,Mosh Hamedani <support@codewithmosh.com>,🚨 Final Hours – Sale Ends Tonight!,2025-02-27 15:14:03+00:00,Last Call! Sale Ends in a Few Hours ‌ ‌ ‌ ‌ ‌ ...,codewithmosh.com 🚨 Final Hours – Sale Ends Ton...,PROMOTION
4,195ce018993e9741,HackerRank Team <no-reply@hackerrankmail.com>,"Improve your Coding Skills with Say ""Hello, Wo...",2025-03-25 15:52:41+00:00,"Hi Varun reddy, Improve your skills with this ...",hackerrankmail.com Improve your Coding Skills ...,PROMOTION


In [18]:
dd['date'].max()

'2025-12-21 14:09:01+00:00'